<a href="https://colab.research.google.com/github/pratikiiitb2013/NLP_END_P1/blob/main/Session9/END_Session9_Seq2Seq_using_RNN_Encoder_Decoder(Cornell_movie_dialogues_corpus).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import ast

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Cornell Movie dialogue corpus

## About the dataset - 
This corpus contains a large metadata-rich collection of fictional conversations extracted from raw movie scripts:



- 220,579 conversational exchanges between 10,292 pairs of movie characters

- involves 9,035 characters from 617 movies

- in total 304,713 utterances

- movie metadata included:

    - genres

    - release year

    - IMDB rating

    - number of IMDB votes

    - IMDB rating

- character metadata included:

    - gender (for 3,774 characters)

    - position on movie credits (3,321 characters)

- see README.txt (included) for details

## Downloading and extracting the dataset

In [2]:
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip cornell_movie_dialogs_corpus.zip
!ls /content/cornell\ movie-dialogs\ corpus

--2021-01-05 12:02:10--  http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9916637 (9.5M) [application/zip]
Saving to: ‘cornell_movie_dialogs_corpus.zip’

cornell_movie_dialo 100%[===================>]   9.46M  12.8MB/s    in 0.7s    

2021-01-05 12:02:11 (12.8 MB/s) - ‘cornell_movie_dialogs_corpus.zip’ saved [9916637/9916637]

Archive:  cornell_movie_dialogs_corpus.zip
   creating: cornell movie-dialogs corpus/
  inflating: cornell movie-dialogs corpus/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/cornell movie-dialogs corpus/
  inflating: __MACOSX/cornell movie-dialogs corpus/._.DS_Store  
  inflating: cornell movie-dialogs corpus/chameleons.pdf  
  inflating: __MACOSX/cornell movie-dialogs corpus/._chameleons.pdf  
  inflating: cornell movie-

## Preprocessing data for model input

#### dataset included different conversation exchanges( can be more than 2 dialogues in a single exchange). So we will be processing dataset such that each consecutive pair of dialogues will be source and target dialogues

In [3]:
df_movie_lines=pd.read_csv("/content/cornell movie-dialogs corpus/movie_lines.txt", 
                           sep = '\+\+\+\$\+\+\+', names = ['lineID','characterID','movieID','char_name','txt'])

df_movie_conversations=pd.read_csv("/content/cornell movie-dialogs corpus/movie_conversations.txt", 
                                   sep='\+\+\+\$\+\+\+',names= ['charID1','charID2','movieID','list_txt'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


In [4]:
print(df_movie_lines.shape)
df_movie_lines.dropna(inplace=True)
df_movie_lines.lineID=df_movie_lines['lineID'].apply(lambda x : x.strip())
df_movie_lines.txt=df_movie_lines['txt'].apply(lambda x : x.strip())
print(df_movie_lines.shape)
df_movie_lines.head()


(304713, 5)
(304446, 5)


,lineID,characterID,movieID,char_name,txt
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [5]:
print(df_movie_conversations.shape)
print(df_movie_conversations.info())
df_movie_conversations.head()

(83097, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83097 entries, 0 to 83096
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   charID1   83097 non-null  object
 1   charID2   83097 non-null  object
 2   movieID   83097 non-null  object
 3   list_txt  83097 non-null  object
dtypes: object(4)
memory usage: 2.5+ MB
None


,charID1,charID2,movieID,list_txt
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [6]:
df_movie_conversations['list_txt'] = df_movie_conversations['list_txt'].apply(lambda x : ast.literal_eval(x.strip()))
df_movie_conversations['dialogue_in_pairs'] = df_movie_conversations['list_txt'].apply(lambda x : list(zip(x, x[1:])))
df_movie_conversations = pd.DataFrame(
      np.concatenate(df_movie_conversations['dialogue_in_pairs'].values), 
      columns = ['d1','d2'])
print(df_movie_conversations.shape)
df_movie_conversations.head()

(221616, 2)


,d1,d2
0,L194,L195
1,L195,L196
2,L196,L197
3,L198,L199
4,L200,L201


In [7]:
df_movie_conversations = df_movie_conversations.merge(
    df_movie_lines[['lineID','txt']], how = 'left', left_on= 'd1', right_on = 'lineID'
    ).merge(
        df_movie_lines[['lineID','txt']], how = 'left', left_on= 'd2', right_on = 'lineID'
    ).rename(columns={"txt_x": "dialogue1", "txt_y": "dialogue2"})[['dialogue1','dialogue2']].dropna().reset_index(drop = True)
print(df_movie_conversations.shape)
df_movie_conversations.head()

(221282, 2)


,dialogue1,dialogue2
0,Can we make this quick? Roxanne Korrine and A...,"Well, I thought we'd start with pronunciation,..."
1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
3,You're asking me out. That's so cute. What's ...,Forget it.
4,"No, no, it's my fault -- we didn't have a prop...",Cameron.


In [8]:
df_movie_conversations = df_movie_conversations.sample(n = 50000, random_state = random.seed(SEED)) 
df_movie_conversations = df_movie_conversations.reset_index(drop = True)

### We have divided conversation history into pairs of dialogues(dialogue1 and dialogue2). Now we will input these pairs seq2seq model

In [9]:
SRC = Field(tokenize='spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = 'spacy', 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [10]:
fields = [('dialogue1', SRC),('dialogue2',TRG)]

In [11]:
example = [data.Example.fromlist([df_movie_conversations.dialogue1[i],df_movie_conversations.dialogue2[i]], fields) for i in range(df_movie_conversations.shape[0])] 

In [12]:
movieDialogueDataset = data.Dataset(example, fields)

In [13]:
(train_data, valid_data, test_data) = movieDialogueDataset.split(split_ratio=[0.70, 0.10, 0.20], random_state=random.seed(SEED))

In [14]:
(len(train_data), len(valid_data), len(test_data))

(35000, 10000, 5000)

In [15]:
print(vars(train_data.examples[1]))
print(vars(train_data.examples[0]))

{'dialogue1': ['captain', 'who', '?'], 'dialogue2': ['captain', 'howdy', '.']}
{'dialogue1': ['do', "n't", 'touch', 'him', '.'], 'dialogue2': ['graham', '.']}


In [16]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [17]:
print('Size of input vocab : ', len(SRC.vocab))
print('Size of label vocab : ', len(TRG.vocab))
print('Top 10 words appreared repeatedly :', list(SRC.vocab.freqs.most_common(10)))

Size of input vocab :  9826
Size of label vocab :  9870
Top 10 words appreared repeatedly : [('.', 35745), (',', 18780), ('you', 17307), ('i', 15552), ('?', 14035), ('the', 11162), ('to', 8919), (' ', 8762), ('a', 7824), ("'s", 7139)]


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [19]:
BATCH_SIZE = 32

# train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
#     (train_data, valid_data, test_data), 
#     batch_size = BATCH_SIZE, 
#     device = device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.dialogue1),
                                                            sort_within_batch=True, device = device)

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [24]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9826, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(9870, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=9870, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,837,518 trainable parameters


In [26]:
optimizer = optim.Adam(model.parameters())

In [27]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [28]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.dialogue1
        trg = batch.dialogue2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [29]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.dialogue1
            trg = batch.dialogue2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')



Epoch: 01 | Time: 5m 17s
	Train Loss: 5.388 | Train PPL: 218.853
	 Val. Loss: 5.541 |  Val. PPL: 254.810
Epoch: 02 | Time: 5m 20s
	Train Loss: 5.036 | Train PPL: 153.870
	 Val. Loss: 5.560 |  Val. PPL: 259.751
Epoch: 03 | Time: 5m 19s
	Train Loss: 4.913 | Train PPL: 136.038
	 Val. Loss: 5.528 |  Val. PPL: 251.569
Epoch: 04 | Time: 5m 21s
	Train Loss: 4.825 | Train PPL: 124.580
	 Val. Loss: 5.558 |  Val. PPL: 259.391
Epoch: 05 | Time: 5m 21s
	Train Loss: 4.686 | Train PPL: 108.455
	 Val. Loss: 5.639 |  Val. PPL: 281.076
Epoch: 06 | Time: 5m 21s
	Train Loss: 4.542 | Train PPL:  93.902
	 Val. Loss: 5.732 |  Val. PPL: 308.652
Epoch: 07 | Time: 5m 20s
	Train Loss: 4.338 | Train PPL:  76.592
	 Val. Loss: 5.869 |  Val. PPL: 353.828
Epoch: 08 | Time: 5m 20s
	Train Loss: 4.160 | Train PPL:  64.069
	 Val. Loss: 5.980 |  Val. PPL: 395.295
Epoch: 09 | Time: 5m 17s
	Train Loss: 3.987 | Train PPL:  53.877
	 Val. Loss: 6.069 |  Val. PPL: 432.068
Epoch: 10 | Time: 5m 21s
	Train Loss: 3.867 | Train PPL

In [32]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.506 | Test PPL: 246.261 |
